# Task 2: Does the test Stimulus (independent variable) have a significant influence on speech quality ratings (dependent variable)? If yes, for which cases? Please assume that each of the six files was assessed by a different set of participants. Use the quality ratings provided in. the data set speech_quality_repetition_dataset

### Step 1: Import libraries and read in data set

In [1]:
# install.packages('dplyr')      # processing 
# install.packages('gdata')      # file reading
# install.packages('car')        # homogenity of variances
# install.packages('rstatix')    # ANOVA

In [2]:
library(dplyr)     # processing
library(readxl)    # reading in data
library(car)       # homogenity of variances
library(rstatix)   # ANOVA

Warning message:
"package 'dplyr' was built under R version 3.6.2"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'car' was built under R version 3.6.2"Loading required package: carData

Attaching package: 'car'

The following object is masked from 'package:dplyr':

    recode

Warning message:
"package 'rstatix' was built under R version 3.6.2"
Attaching package: 'rstatix'

The following object is masked from 'package:stats':

    filter



In [3]:
use_ratings_mean <- TRUE

In [4]:
# read in data sets
get_quality_data <- function() {
    quality_data <- read.csv("datasets/DB03_speech_quality_repetition_dataset.csv")
    # use mean here as indicator of given rating per participant
    if(use_ratings_mean) {
        quality_data <- quality_data %>% 
                        rename(Rating = rating) %>%
                        select(subjectCode, testStimulus, Rating, repetition) %>%
                        group_by(testStimulus, subjectCode) %>%
                        summarize(Rating = mean(Rating)) %>%
                        select(testStimulus, Rating)
    } else {
        quality_data <- quality_data %>% 
                        rename(Rating = rating) %>%
                        select(subjectCode, testStimulus, Rating) 
    }
    
    quality_data
}

quality_data <- get_quality_data()
head(quality_data)

testStimulus,Rating
haus_m_700_bpf_200_2800_normAsl_-26,2.00
haus_m_700_bpf_200_2800_normAsl_-26,2.50
haus_m_700_bpf_200_2800_normAsl_-26,2.50
haus_m_700_bpf_200_2800_normAsl_-26,3.75
haus_m_700_bpf_200_2800_normAsl_-26,2.00
haus_m_700_bpf_200_2800_normAsl_-26,1.00


### Step 2: Decide on which ANOVA test to use

#### => 1 independent input variables (testStimulus), 1 dependent variable, NOT repeated measures / independent => one-way independent measure ANOVA

### Step 3: Check assumptions

#### 1. Dependent variables on interval or ratio scale => check, because rating is discrete & on interval scale
#### 2. Independent variables with two or more groups => check, because testStimulus has multiple different values 
#### 3. Indepenence of observation => check, because observations come from different people, therefore independence can be assumed
#### 4. No significant outliers => don't know, need to check that in the next step!
#### 5. Normally distributed population for every single group => don't know, need to check that in the next step!
#### 6. Homogenity of variances => don't know, need to check that in the next step!

### Step 3.1: Outlier detection

In [5]:
# z score method
if(use_ratings_mean) {
    z_scores <- quality_data %>% 
                    mutate(Std_Dev_Rating = sd(Rating), 
                           Mean_Rating = mean(Rating)) %>%
                    mutate(Z_Score_Rating = (Rating - Mean_Rating) / Std_Dev_Rating) %>%
                    select(testStimulus, Rating, Z_Score_Rating) %>%
                    drop_na() %>%
                    arrange(desc(Z_Score_Rating))
} else {
    z_scores <- merge(
                    quality_data %>% 
                                group_by(testStimulus) %>%
                                summarize(Std_Dev_Rating = sd(Rating), 
                                       Mean_Rating = mean(Rating)), 
                     (quality_data %>% select(testStimulus, Rating))) %>%
                drop_na() %>%
                mutate(Z_Score_Rating = (Rating - Mean_Rating) / Std_Dev_Rating) %>%
                arrange(desc(Z_Score_Rating))
}

head(z_scores)

testStimulus,Rating,Z_Score_Rating
maus_m_700_bpf_200_2800_normAsl_-26,5.00,2.710246
haus_m_700_bpf_200_2800_normAsl_-26,5.00,2.472542
maus_m_700_bpf_200_2800_normAsl_-26,4.75,2.419582
haus_m_700_bpf_200_2800_normAsl_-26,4.75,2.209657
haus_m_700_mnru_Q_14_normAsl_-26,3.75,2.159806
maus_m_700_mnru_Q_14_normAsl_-26,3.50,2.068097


#### Criterion checked: no significant outliers (no absolute z score greater than 3.29)

### Step 3.2: Normally distributed population for every single group

In [6]:
# normality checking for groups
check_normality_for_group <- function(stimulus) {
    data <- quality_data %>% dplyr::filter(testStimulus == stimulus) 
    test_result <- ks.test(data[['Rating']], "pnorm", mean=mean(data[['Rating']]), sd=sd(data[['Rating']]))
    result_string <- paste0('Normality for testStimulus ', stimulus, ': ')
    test_result[['p.value']]
    
    if(test_result[['p.value']] < 0.05) {
        result_string <- paste0(result_string, as.character(round(test_result[['p.value']], digits=7)), 
                                ' p-value (Kolmogrov-Smirnov) == NO, ')
    } else {
        result_string <- paste0(result_string, as.character(round(test_result[['p.value']], digits=7)), 
                                ' p-value (Kolmogrov-Smirnov) == YES, ')
    }
    
    test_result <- shapiro.test(data[['Rating']])
    if(test_result[['p.value']] < 0.05) {
        result_string <- paste0(result_string, as.character(round(test_result[['p.value']], digits=7)), 
                                ' p-value (Shapiro-Wilk) == NO!')
    } else {
        result_string <- paste0(result_string, as.character(round(test_result[['p.value']], digits=7)), 
                                ' p-value (Kolmogrov-Smirnov) == YES!')
    }
    
    result_string
}

In [7]:
# print normality tests
stimuli <- (quality_data %>% distinct(testStimulus))[['testStimulus']]

for (stimulus in stimuli) {
    print(check_normality_for_group(stimulus))
}

Warning message in ks.test(data[["Rating"]], "pnorm", mean = mean(data[["Rating"]]), :
"ties should not be present for the Kolmogorov-Smirnov test"

[1] "Normality for testStimulus haus_m_700_bpf_200_2800_normAsl_-26: 0.7356822 p-value (Kolmogrov-Smirnov) == YES, 0.2041441 p-value (Kolmogrov-Smirnov) == YES!"


Warning message in ks.test(data[["Rating"]], "pnorm", mean = mean(data[["Rating"]]), :
"ties should not be present for the Kolmogorov-Smirnov test"

[1] "Normality for testStimulus haus_m_700_mnru_Q_14_normAsl_-26: 0.2304167 p-value (Kolmogrov-Smirnov) == YES, 0.084306 p-value (Kolmogrov-Smirnov) == YES!"


Warning message in ks.test(data[["Rating"]], "pnorm", mean = mean(data[["Rating"]]), :
"ties should not be present for the Kolmogorov-Smirnov test"

...

[1] "Normality for testStimulus maus_m_700_bpf_200_2800_normAsl_-26: 0.5205064 p-value (Kolmogrov-Smirnov) == YES, 0.0851928 p-value (Kolmogrov-Smirnov) == YES!"


Warning message in ks.test(data[["Rating"]], "pnorm", mean = mean(data[["Rating"]]), :
"ties should not be present for the Kolmogorov-Smirnov test"

[1] "Normality for testStimulus maus_m_700_mnru_Q_14_normAsl_-26: 0.1511504 p-value (Kolmogrov-Smirnov) == YES, 0.0208674 p-value (Shapiro-Wilk) == NO!"


Warning message in ks.test(data[["Rating"]], "pnorm", mean = mean(data[["Rating"]]), :
"ties should not be present for the Kolmogorov-Smirnov test"

[1] "Normality for testStimulus maus_m_700_normAsl_-26: 0.0280163 p-value (Kolmogrov-Smirnov) == NO, 2e-07 p-value (Shapiro-Wilk) == NO!"


#### Normality is hence likely to not exist within all groups - we are still going to continue though.

### Step 3.3: Homogenity of variances

In [8]:
# Check for homogenity of groups' VQ ratings
get_levene_test_results <- function() {

    test_results <- leveneTest(Rating ~ testStimulus, data = quality_data, center = mean)
    test_results

    stimuli <- (quality_data %>% distinct(testStimulus))[['testStimulus']]
    result <- 'F('
    for (stimulus in stimuli) {
        df <- (quality_data %>% dplyr::filter(testStimulus == stimulus) %>% 
               mutate(df = n() - 1))[1,][['df']]
        result <- paste0(result, ' df_{', stimulus, '} = ', df, ';')
    }
    result <- paste0(result, ' ) = ', 
                     round(test_results[1,2], digits=7), 
                ' | p-value = ', 
                round(test_results[1,3], digits=7))
    
    if(test_results[1,3] > 0.05) {
        result <- paste0(result, ' => homogenity of variance CAN be assumed')
    } else {
        result <- paste0(result, ' => homogenity of variance CANNOT be assumed')
    }
    
    result
}

get_levene_test_results()

[1] "F( df_{haus_m_700_bpf_200_2800_normAsl_-26} = 36; df_{haus_m_700_mnru_Q_14_normAsl_-26} = 36; df_{haus_m_700_normAsl_-26} = 36; df_{maus_m_700_bpf_200_2800_normAsl_-26} = 36; df_{maus_m_700_mnru_Q_14_normAsl_-26} = 36; df_{maus_m_700_normAsl_-26} = 36; ) = 2.7341014 | p-value = 0.0203379 => homogenity of variance CANNOT be assumed"

#### As Levene's test delivers p-value < 0.05 => homogenity of variance can NOT be assumed, but we are going to continue anyways!

### Step 4: Conduct one-way independent measure ANOVA

In [9]:
# conduct one-way independent measure ANOVA
anova_res <- aov(Rating ~ testStimulus, data = quality_data)
anova_res

anova_summarized <- anova_summary(anova_res, detailed = TRUE)
anova_summarized

Call:
   aov(formula = Rating ~ testStimulus, data = quality_data)

Terms:
                testStimulus Residuals
Sum of Squares      235.6737  117.5980
Deg. of Freedom            5       216

Residual standard error: 0.7378584
Estimated effects may be unbalanced

Effect,DFn,DFd,SSn,SSd,F,p,p<.05,ges
testStimulus,5,216,235.674,117.598,86.576,1.2e-49,*,0.667


#### Note: testStimulus has significant effect on speech quality ratings (more on that later)

### Step 5: Pairwise comparison / post hoc test

In [10]:
# as equal sample sizes (each n=148) and Tukey most widely used: Tukey ["Cramming Sam's tips" for post hoc tests (from lecture)]
post_hoc <- TukeyHSD(anova_res)
comparisons <- tibble::rownames_to_column(as.data.frame(post_hoc$testStimulus), "pairwise comparison") %>% 
                select(`pairwise comparison`, `p adj`) %>%
                arrange(desc(`p adj`))
paste0('Tukey:')
comparisons

# BONUS: Bonferroni because we want to control Type I error
paste0('Bonferroni (bonus):')
x <- pairwise.t.test(quality_data$Rating, quality_data$testStimulus, p.adj="bonferroni")
x$p.value

[1] "Tukey:"

pairwise comparison,p adj
maus_m_700_bpf_200_2800_normAsl_-26-haus_m_700_bpf_200_2800_normAsl_-26,0.999996745
maus_m_700_normAsl_-26-haus_m_700_normAsl_-26,0.985074924
maus_m_700_mnru_Q_14_normAsl_-26-haus_m_700_mnru_Q_14_normAsl_-26,0.985074924
haus_m_700_mnru_Q_14_normAsl_-26-haus_m_700_bpf_200_2800_normAsl_-26,0.083836735
maus_m_700_bpf_200_2800_normAsl_-26-haus_m_700_mnru_Q_14_normAsl_-26,0.061835383
maus_m_700_mnru_Q_14_normAsl_-26-haus_m_700_bpf_200_2800_normAsl_-26,0.012192289
maus_m_700_mnru_Q_14_normAsl_-26-maus_m_700_bpf_200_2800_normAsl_-26,0.008264769
haus_m_700_normAsl_-26-haus_m_700_bpf_200_2800_normAsl_-26,0.000000000
maus_m_700_normAsl_-26-haus_m_700_bpf_200_2800_normAsl_-26,0.000000000
haus_m_700_normAsl_-26-haus_m_700_mnru_Q_14_normAsl_-26,0.000000000


[1] "Bonferroni (bonus):"

,haus_m_700_bpf_200_2800_normAsl_-26,haus_m_700_mnru_Q_14_normAsl_-26,haus_m_700_normAsl_-26,maus_m_700_bpf_200_2800_normAsl_-26,maus_m_700_mnru_Q_14_normAsl_-26
haus_m_700_mnru_Q_14_normAsl_-26,1.195297e-01,NA,NA,NA,NA
haus_m_700_normAsl_-26,2.450942e-20,9.554508e-29,NA,NA,NA
maus_m_700_bpf_200_2800_normAsl_-26,1.000000e+00,8.448747e-02,5.623455e-20,NA,NA
maus_m_700_mnru_Q_14_normAsl_-26,1.440882e-02,1.000000e+00,6.889294e-31,9.556395e-03,NA
maus_m_700_normAsl_-26,2.110953e-22,6.889294e-31,1.000000e+00,4.912038e-22,4.920252e-33


#### => More: see interpretation

### Step 6: Interpretation

In [11]:
# compute individual degrees of freedom for groups
get_df_of <- function(stimulus) {
    df <- (quality_data %>% dplyr::filter(testStimulus == stimulus) %>% 
            mutate(df = n() - 1))[1,][['df']]
    paste0('df_{', stimulus, '} = ', df, '; ')
}

paste0('Total amount of samples: ', nrow(quality_data))

[1] "Total amount of samples: 222"

In [12]:
# compute statistics for independent variables' values
for (stimulus in stimuli) {
    statistics <- quality_data %>% 
                    filter(testStimulus == stimulus) %>%
                    group_by(testStimulus) %>%
                    summarize(mean = mean(Rating), sd = sd(Rating))
    print(paste0(stimulus, ' (', get_df_of(stimulus), '): mean around ', round(statistics[['mean']], digits=3), ', standard deviation around ', round(statistics[['sd']], digits=3)))
}

[1] "haus_m_700_bpf_200_2800_normAsl_-26 (df_{haus_m_700_bpf_200_2800_normAsl_-26} = 36; ): mean around 2.649, standard deviation around 0.951"
[1] "haus_m_700_mnru_Q_14_normAsl_-26 (df_{haus_m_700_mnru_Q_14_normAsl_-26} = 36; ): mean around 2.189, standard deviation around 0.723"
[1] "haus_m_700_normAsl_-26 (df_{haus_m_700_normAsl_-26} = 36; ): mean around 4.473, standard deviation around 0.533"
[1] "maus_m_700_bpf_200_2800_normAsl_-26 (df_{maus_m_700_bpf_200_2800_normAsl_-26} = 36; ): mean around 2.669, standard deviation around 0.86"
[1] "maus_m_700_mnru_Q_14_normAsl_-26 (df_{maus_m_700_mnru_Q_14_normAsl_-26} = 36; ): mean around 2.074, standard deviation around 0.689"
[1] "maus_m_700_normAsl_-26 (df_{maus_m_700_normAsl_-26} = 36; ): mean around 4.588, standard deviation around 0.584"


#### Altering the testStimulus does have a significant effect on the speech quality ratings: 
#### There is a significant (alpha value = 0.05) and rather large main effect of the testStimulus on the speech quality ratings (F statistic value of around 86.576, p-value of around 1.2e-49 with eta² effect size of around 0.667). 
#### Overall residual standard error around 0.738 with sum of squares around 117.6.
#### The total degrees of freedom are the amount of total observations - 1 => 222 - 1 = 221 [37 samples & 36 df for each distinct testStimulus]. 
#### Regarding pairwise comparisons / post hoc tests, there is a significant statistical difference of ratings [with p-value < 0.05] between every pair of different testStimuli except for 5 cases: 
#### maus_m_700_bpf_200_2800_normAsl_-26 to haus_m_700_bpf_200_2800_normAsl_-26
#### maus_m_700_normAsl_-26 to haus_m_700_normAsl_-26
#### maus_m_700_mnru_Q_14_normAsl_-26 to haus_m_700_mnru_Q_14_normAsl_-26
#### haus_m_700_mnru_Q_14_normAsl_-26 to haus_m_700_bpf_200_2800_normAsl_-26
#### maus_m_700_bpf_200_2800_normAsl_-26 to haus_m_700_mnru_Q_14_normAsl_-26